In [1]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

In [ ]:
def extract_info(url, wait_time = 3, delay_time = 1):
    data={}
    max_retries = 3
    for attempt in range(max_retries):
        try:
            chrome_options = Options()
            chrome_options.add_argument('--headless')  # Headless 모드 활성화
            driver = webdriver.Chrome(options=chrome_options)
            driver.implicitly_wait(wait_time)
            driver.get(url)
            html = driver.page_source
            time.sleep(delay_time)
            driver.quit()
            
            soup = BeautifulSoup(html,'html.parser')
           
            title = soup.find('h3', {'class': 'tit_subject'}).get_text(strip=True)
            user_id = soup.find('span', {'class':'txt_subject'}).split('|')[0].replace('작성자','').strip()
            post_time = soup.find('span', {'class':'txt_subject'}).split('|')[1].replace('작성시간','').strip()
            post_time = datetime.strptime(post_time, '%Y-%m-%d')
            post = soup.find(id='article').get_text(strip=True)
            view_cnt = soup.find('span', {'class':'txt_subject'}).split('|')[2].replace('조회수','').strip()
            reply_cnt = int(soup.find(class_="num_total").get_text(strip=True))
            reply_content = []
            if reply_cnt != 0:
                comment_button = driver.find_element(By.CLASS_NAME, 'link_all')
                comment_button.click()
                
                # 페이지 내의 코멘트들을 모두 찾자
                cmt_all = soup.body.find('ul', id = 'commentList').find_all('li')
    
                # 찾은 코멘트들을 언패킹하자
                for k in cmt_all:
                    if k.div.span.find_all('span', class_ = 'txt_bar'):
                        if k['class'] == ['reply_on']:
                            cmt_reply = True
                        else:
                            cmt_reply = False
                            
                        cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                        cmt_time = k.div.span.find('span', class_='num_info').get_text()
                        cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                        reply_content.append({'author': cmt_writer, 'text': cmt_txt, 'time' : cmt_time})
            data = {'title': title, 'user_id': user_id, 'post_time': post_time, 'post': post, 'view_cnt': view_cnt, 'reply_cnt': reply_cnt, 'reply_content': reply_content}
            
            # 댓글이 있다면 딕셔너리에 추가
            if reply_cnt != 0:
                for i, reply_info in enumerate(reply_content):
                    data[f'reply_{i+1}_author'] = reply_info['author']
                    data[f'reply_{i+1}_text'] = reply_info['text']
                    data[f'reply_{i+1}_time'] = reply_info['time']
            print(url, '완료')
            break  # 성공적으로 크롤링이 완료되면 루프 종료
        except Exception as e:
            print(f"Crawling failed, retrying... ({attempt+1}/{max_retries})")
            time.sleep(2)  # 2초 대기 후 재시도
    else:
        print("Crawling failed after maximum retries.")
    
    return data

extract_info('https://m.cafe.daum.net/rocksoccer/ADs2/488585')

Crawling failed, retrying... (1/5)
Crawling failed, retrying... (2/5)
Crawling failed, retrying... (3/5)
Crawling failed, retrying... (4/5)
Crawling failed, retrying... (5/5)
Crawling failed after maximum retries.


{}

In [26]:
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.
#    AJAX를 사용한 페이지는 페이지 전환시 딜레이가 꼭 필요한 경우도 있습니다.

In [32]:
def extract_info(url, wait_time=3, delay_time=1):
    max_retries = 2
    for attempt in range(max_retries):
        try:
            chrome_options = Options()
            # chrome_options.add_argument('--headless')  # Headless 모드 활성화
            # 원하는 User-Agent 설정
            driver.get(url)
            html = driver.page_source
            time.sleep(delay_time)
            soup = BeautifulSoup(html, 'html.parser')

            title = soup.find('h3', class_= 'tit_subject').get_text(strip=True)
            user_info = soup.find('span', {'class': 'txt_subject'}).get_text("|", strip=True).split('|')
            user_id = user_info[0].replace('작성자', '').strip()
            post_time = user_info[1].replace('작성시간', '').strip()
            post_time = datetime.strptime(post_time, '%Y-%m-%d')
            post = soup.find(id='article').get_text(strip=True)
            view_cnt = user_info[2].replace('조회수', '').strip()
            reply_cnt = int(soup.find(class_="num_total").get_text(strip=True))
            reply_content = []

            print(f"title: {title}")
            print(f"user_id: {user_id}")
            print(f"post_time: {post_time}")
            print(f"post: {post}")
            print(f"view_cnt: {view_cnt}")
            print(f"reply_cnt: {reply_cnt}")

            if reply_cnt != 0:
                comment_button = driver.find_element(By.CLASS_NAME, 'link_all')
                comment_button.click()
                time.sleep(1)  # 동적 콘텐츠 로딩 대기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                cmt_all = soup.body.find('ul', id='commentList').find_all('li')

                for k in cmt_all:
                    if k.div.span.find_all('span', class_='txt_bar'):
                        if 'reply_on' in k['class']:
                            cmt_reply = True
                        else:
                            cmt_reply = False

                        cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                        cmt_time = k.div.span.find('span', class_='num_info').get_text()
                        cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                        reply_content.append({'author': cmt_writer, 'text': cmt_txt, 'time': cmt_time})
                        print(f"comment: {cmt_writer}, {cmt_txt}, {cmt_time}")

            driver.quit()

            print(url, '완료')
            break  # 성공적으로 크롤링이 완료되면 루프 종료
        except Exception as e:
            print(f"Crawling failed, retrying... ({attempt + 1}/{max_retries})")
            
            time.sleep(2)  # 2초 대기 후 재시도
    else:
        print("Crawling failed after maximum retries.")


In [33]:
extract_info('https://m.cafe.daum.net/rocksoccer/ADs2/488585')

Crawling failed, retrying... (1/2)
Crawling failed, retrying... (2/2)
Crawling failed after maximum retries.


In [33]:
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.
#    AJAX를 사용한 페이지는 페이지 전환시 딜레이가 꼭 필요한 경우도 있습니다.

In [38]:
def extract_info(url, wait_time=3, delay_time=1):
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # Headless 모드 활성화
    # 원하는 User-Agent 설정
    driver.get(url)
    html = driver.page_source
    time.sleep(delay_time)
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h3', class_= 'tit_subject').get_text(strip=True)
    user_info = soup.find('span', {'class': 'txt_subject'}).get_text("|", strip=True).split('|')
    user_id = user_info[1].replace('작성자', '').strip()
    post_time = soup.find(class_='num_subject').contents[0].strip()
    post = soup.find(id='article').get_text(strip=True)
    view_cnt = soup.find_all(class_='num_subject')[1].text.strip()
    reply_cnt = int(soup.find(class_="num_total").get_text(strip=True))
    print(reply_cnt)
    reply_content = []

    if reply_cnt != 0:
        # 댓글 페이지로 이동

        driver.get('https://m.cafe.daum.net/rocksoccer/ADs2/488585/comments')
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 페이지 숫자 확인
        cmt_page_max = int(soup.body.find('span', id="pagingNav").find_all('span', class_="num_page").pop().get_text())
        print('cmt_page_max', cmt_page_max)
    
        # 페이지를 하나씩 넘기면서 캡쳐.. 
    
        for i in range(cmt_page_max):
            j = cmt_page_max-i
        
            # 캡처
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
        
            # 페이지 내의 코멘트들을 모두 찾자
            cmt_all = soup.body.find('ul', id = 'commentList').find_all('li')
        
            # 찾은 코멘트들을 언패킹하자
            for k in cmt_all:
                if k.div.span.find_all('span', class_ = 'txt_bar'):
                    if k['class'] == ['reply_on']:
                        cmt_reply = True
                        print('ㄴ', end='') #대댓글을 따로 처리? 걍 또다른 댓글이라고 치자..
                    else:
                        cmt_reply = False
                    
                    cmt_num = inp_num +'_'+ k['id'].split('comment_')[1]
                    cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                    cmt_time = k.div.span.find('span', class_='num_info').get_text()
                    cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                
                    # 언패킹한 결과물을 화면 출력
                    print(cmt_num, '[', cmt_writer,']', cmt_time, cmt_txt)

            # 페이지가 0으로 넘어가지 않도록 if문 작성
        
            if j > 1:
                #페이지 이동
                driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page = {3}&mode=regular&depth={4}&page={5}'.format(CAFE_NAME, BOARD_NAME, inp_num, j, '0002100000', (j-1)))
                time.sleep(2)
    print(url, '완료')


In [39]:
extract_info('https://m.cafe.daum.net/rocksoccer/ADs2/488585')

21
cmt_page_max 2


NameError: name 'CAFE_NAME' is not defined

In [32]:
reply_cnt

NameError: name 'reply_cnt' is not defined